## 1. Installation

## 2.Writer

To see python format, check https://python.readthedocs.io/en/v2.7.2/library/struct.html#format-characters

In [1]:
czip Writer -- --help

NAME
    czip Writer

SYNOPSIS
    czip Writer COMMAND | <flags>

FLAGS
    -O, --Output=OUTPUT
        Type: Optional[]
        Default: None
    --mode=MODE
        Default: 'wb'
    -F, --Formats=FORMATS
        Default: ['H', 'H']
    -C, --Columns=COLUMNS
        Default: ['mc', 'cov']
    -D, --Dimensions=DIMENSIONS
        Default: ['chrom']
    -f, --fileobj=FILEOBJ
        Type: Optional[]
        Default: None
    --message=MESSAGE
        Default: ''
    -l, --level=LEVEL
        Default: 6
    -v, --verbose=VERBOSE
        Default: 0

COMMANDS
    COMMAND is one of the following:

     create_new_dim


In [2]:
czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H -- --help

NAME
    czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H

SYNOPSIS
    czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H - GROUP | COMMAND | VALUE

GROUPS
    GROUP is one of the following:

     Columns

     Dimensions

     Formats

COMMANDS
    COMMAND is one of the following:

     catcz
       Cat multiple .cz files into one .cz file.

     close
       Flush data, write 28 bytes BGZF EOF marker, and close BGZF file.

     create_new_dim

     fileno
       Return integer file descriptor.

     flush
       Flush data explicitally.

     isatty
       Return True if connected to a TTY device.

     seekable

     tell
       Return a BGZF 64-bit virtual offset.

     tocz
       Pack dataframe, stdin or a file path into .cz file with or without reference coordinates file.

     write_chunk
       Write data into one chunk with dimension name (such as ['cell1','chr1'])

     write_header

VALUES
    VALUE is one of the followin

In [3]:
czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H tocz --help

INFO: Showing help with the command 'czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H tocz -- --help'.

NAME
    czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H tocz - Pack dataframe, stdin or a file path into .cz file with or without reference coordinates file.

SYNOPSIS
    czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H tocz <flags>

DESCRIPTION
    Pack dataframe, stdin or a file path into .cz file with or without reference coordinates file.

FLAGS
    -I, --Input=INPUT
        Type: Optional[]
        Default: None
        list, tuple, np.ndarray or dataframe, stdin (Input is None, "stdin" or "-"), or a file path (need to specify sep,header and skiprows).
    -u, --usecols=USECOLS
        Default: [4, 5]
        usecols is the index of columns to be packed into .cz Columns.
    -d, --dim_cols=DIM_COLS
        Default: [0]
        index of columns to be set as Dimensions of Writer, such as chrom colu

In [6]:
cat test1.bed

chr1	3001733	+	CTG	0	1	1
chr1	3001743	+	CAA	0	1	1
chr1	3001746	+	CCT	0	1	1
chr1	3001747	+	CTG	0	1	1
chr8	3002093	-	CTG	0	1	1
chr8	3002098	-	CAA	0	1	1
chr8	3002101	-	CAG	0	1	1
chr8	3002102	-	CCA	0	1	1
chr8	3002103	-	CCC	0	1	1


In [7]:
czip Writer -O test1.mz -D chrom -C pos,strand,context,mc,cov -F Q,c,3s,H,H tocz -I test1.bed -u 1,2,3,4,5

# 2. Reader

view test1.mz

In [8]:
czip Reader -I test1.mz view -s 0 test1.mz

chrom	pos	strand	context	mc	cov
chr1	3001733	+	CTG	0	1
chr1	3001743	+	CAA	0	1
chr1	3001746	+	CCT	0	1
chr1	3001747	+	CTG	0	1
chr8	3002093	-	CTG	0	1
chr8	3002098	-	CAA	0	1
chr8	3002101	-	CAG	0	1
chr8	3002102	-	CCA	0	1
chr8	3002103	-	CCC	0	1


To reduct the file size, we can pack columns 2-4 into reference.mz and 5-6 into test1.mz <br>
This will be useful when we have lots of samples or cells sharing the same reference

In [10]:
cut -f 1-4 test1.bed | czip Writer -O ref.mz -D chrom -C pos,strand,context -F Q,c,3s tocz -I stdin -u 1,2,3

In [12]:
cut -f 1,5,6 test1.bed | czip Writer -O test2.mz -D chrom -C mc,cov -F H,H tocz -I stdin -u 1,2

In [11]:
czip Reader -I ref.mz view -s 0

chrom	pos	strand	context
chr1	3001733	+	CTG
chr1	3001743	+	CAA
chr1	3001746	+	CCT
chr1	3001747	+	CTG
chr8	3002093	-	CTG
chr8	3002098	-	CAA
chr8	3002101	-	CAG
chr8	3002102	-	CCA
chr8	3002103	-	CCC


In [14]:
czip Reader -I test2.mz view

mc	cov
0	1
0	1
0	1
0	1
0	1
0	1
0	1
0	1
0	1


view test2.mz together with ref.mz

In [26]:
czip Reader -I test2.mz view -s 0 -r ref.mz

chrom	pos	strand	context	mc	cov
chr1	3001733	+	CTG	0	1
chr1	3001743	+	CAA	0	1
chr1	3001746	+	CCT	0	1
chr1	3001747	+	CTG	0	1
chr8	3002093	-	CTG	0	1
chr8	3002098	-	CAA	0	1
chr8	3002101	-	CAG	0	1
chr8	3002102	-	CCA	0	1
chr8	3002103	-	CCC	0	1


Only view one chrom (dimension)

In [27]:
czip Reader -I test2.mz view -s 0 -r ref.mz -D chr8

chrom	pos	strand	context	mc	cov
chr8	3002093	-	CTG	0	1
chr8	3002098	-	CAA	0	1
chr8	3002101	-	CAG	0	1
chr8	3002102	-	CCA	0	1
chr8	3002103	-	CCC	0	1


# 3. Query

In [29]:
czip Reader -I test2.mz query --help

INFO: Showing help with the command 'czip Reader -I test2.mz query -- --help'.

NAME
    czip Reader -I test2.mz query - query .cz file by Dimension, start and end, if regions provided, Dimension, start and end should be None, regions should be a list, each element of regions is a list, for example, regions=[[('cell1','chr1'),1,10], [('cell10','chr22'),100,200]],and so on.

SYNOPSIS
    czip Reader -I test2.mz query <flags>

DESCRIPTION
    query .cz file by Dimension, start and end, if regions provided, Dimension, start and end should be None, regions should be a list, each element of regions is a list, for example, regions=[[('cell1','chr1'),1,10], [('cell10','chr22'),100,200]],and so on.

FLAGS
    -D, --Dimension=DIMENSION
        Type: Optional[]
        Default: None
        dimension, such as chr1 (if Dimensions 'chrom' is inlcuded in header['Dimensions']), or sample1 (if something like 'sampleID' is included in header['Dimensions'] and chunk contains such dimension)
    -s, --s

In [28]:
czip Reader -I test2.mz query -D chr8 -s 3002101 -e 3002102 -r ref.mz

chrom	pos	strand	context	mc	cov
chr8	3002101	-	CAG	0	1
chr8	3002102	-	CCA	0	1
